In [1]:
import pandas as pd

In [4]:
train_df = pd.read_csv('../../model/datasets/challenge_train.csv')
test_df = pd.read_csv('../../model/datasets/challenge_test.csv')

In [7]:
train_df_ = train_df[['id', 'name', 'mana', 'attack', 'health', 'type', 'god']]

In [9]:
train_df_.columns

Index(['id', 'name', 'mana', 'attack', 'health', 'type', 'god'], dtype='object')

In [10]:
test_df.columns

Index(['id', 'name', 'mana', 'attack', 'health', 'type', 'god'], dtype='object')

In [16]:
all_data = pd.concat([train_df_, test_df])

In [18]:
import joblib

In [23]:
lgb_model = joblib.load('../../model/lgb_model.joblib')

In [29]:
all_data['mana_group'] = pd.cut(all_data['mana'], 5, labels=[0,1,2,3,4])
all_data['attack_group'] = pd.cut(all_data['attack'], 5, labels=[0,1,2,3,4])
all_data['health_group'] = pd.cut(all_data['attack'], 5, labels=[0,1,2,3,4])
gods_c = {'neutral': 0,
          'light': 1,
          'nature': 2,
          'deception': 3,
          'death': 4,
          'war': 5,
          'magic': 6}
all_data['god_categories'] = all_data['god'].map(lambda x: gods_c.get(x, 7))
type_c = {'creature': 0,
          'spell': 1,
          'weapon': 2,
          'god power': 3}
all_data['type_categories'] = all_data['type'].map(lambda x: type_c.get(x, 4))
X = all_data[['type_categories', 'god_categories', 'mana_group', 'attack_group', 'health_group']]

In [30]:
all_data['strategy'] = (lgb_model.predict(X, num_iteration=lgb_model.best_iteration) > 0.5).astype(int)
all_data['strategy'] = all_data.strategy.apply(lambda x: "late" if x == 1 else "early")

In [32]:
all_data[train_df.columns].head()

,id,name,mana,attack,health,type,god,strategy
0,1118,Firewine,5,0,0,spell,nature,early
1,1036,Leyhoard Hatchling,10,2,1,creature,magic,late
2,244,Aetherfuel Alchemist,6,4,4,creature,neutral,late
3,215,Millenium Matryoshka,4,2,2,creature,neutral,late
4,87013,Poison Peddler,4,1,3,creature,neutral,late
